[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RadimKozl/JLNN/blob/main/examples/JLNN_basic_inference.ipynb)

# **JLNN – Basic inference + manual grounding**

This notebook shows the simplest use of the JLNN framework:

- Definition of a logical rule without training
- Manual setting of truth intervals [L, U] for each predicate
- Calculation of the output interval for the entire rule
- Display of the uncertainty width (U - L)

## ***1. Installation and automatic restart***

In [ ]:
try:
    import jlnn
    from flax import nnx
    import jax.numpy as jnp
    print("✅ JLNN and JAX are ready.")
except ImportError:
    print("🚀 Installing JLNN from GitHub and fixing JAX for Colab...")
    # Instalace frameworku
    !pip install jax-lnn --quiet
    # Fix JAX/CUDA compatibility for 2026 in Colab
    !pip install --upgrade "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

    import os
    print("\n🔄 RESTARTING ENVIRONMENT... Please wait a second and then run the cell again.")
    os.kill(os.getpid(), 9)
    os.kill(os.getpid(), 9) # After this line, the cell stops and the environment restarts

🚀 Installing JLNN from GitHub and fixing JAX for Colab...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os
os.environ["JAX_PLATFORMS"] = "cpu"

### ***Imports***

In [2]:
import jax.numpy as jnp
from flax import nnx
from jlnn.symbolic.compiler import LNNFormula

In [3]:
print("JLNN loaded.")

JLNN loaded.


## ***2. Definition of a simple rule***

We will use the classic rule:

```python
0.8::A & B -> C
```

- Rule weight 0.8 (confidence in implication)
- A ∧ B implies C
- No weighted antecedent or temporal operator (yet)

### ***model creation***

In [4]:
# Model creation – compiling rules into an NNX graph
model = LNNFormula("0.8::A & B -> C", nnx.Rngs(42))

In [5]:
print("Model created. Predicates:", list(model.predicates.keys()))

Model created. Predicates: ['A', 'B', 'C']


## ***3. Manual grounding – setting intervals***

Each predicate (A, B, C) gets a **manual truth interval [L, U]**.

- L = lower bound (minimum confirmed truth)
- U = upper bound (maximum possible truth)
- Width of the interval U - L = degree of uncertainty/ignorance

### ***interval setting + inference***

Manually set intervals for each predicate <br>
Format: [L, U] for batch size 1

In [6]:
inputs = {
    "A": jnp.array([[0.7, 0.9]]),   # And it is quite likely
    "B": jnp.array([[0.4, 0.8]]),   # B has more uncertainty
    "C": jnp.array([[0.0, 1.0]])    # C is completely unknown (ignorance)
}

Output interval for C

In [7]:
output = model(inputs)

We will find out the shape and adjust the approach

In [8]:
print("Output shape:", output.shape)

Output shape: (1, 2, 2)


If the batch dimension is (usually the first), we take the first sample

In [9]:
if len(output.shape) == 3:          # (batch, 1, 2) or similar
    L = output[0, 0, 0].item()
    U = output[0, 0, 1].item()
elif len(output.shape) == 2:        # (batch, 2)
    L = output[0, 0].item()
    U = output[0, 1].item()
elif len(output.shape) == 1:        # only (2,)
    L = output[0].item()
    U = output[1].item()
else:
    raise ValueError(f"Unknown output shape: {output.shape}")

In [10]:
print("Output interval for C:")
print(f"  L = {L:.4f}")
print(f"  U = {U:.4f}")
print(f"  Uncertainty (width): {U - L:.4f}")

Output interval for C:
  L = 0.6000
  U = 0.7000
  Uncertainty (width): 0.1000


## ***4. Experiments – different input intervals***

Try changing the values ​​and running the cell again.
What happens if:

- A and B have high L (e.g. [0.9, 1.0]) → C should have high L
- A has low U (e.g. [0.0, 0.2]) → C will be low
- One of the antecedents has high uncertainty → uncertainty is transferred to C

### ***second set of experiments***

Experiment 1: both antecedents strongly true

In [11]:
inputs_exp1 = {
    "A": jnp.array([[0.95, 1.0]]),
    "B": jnp.array([[0.90, 0.98]]),
    "C": jnp.array([[0.0, 1.0]])
}
print("Exp 1 – strong A and B:")
print(model(inputs_exp1))

Exp 1 – strong A and B:
[[[0.5 0.7]
  [1.  1. ]]]


Experiment 2: one antecedent weak

In [12]:
inputs_exp2 = {
    "A": jnp.array([[0.95, 1.0]]),
    "B": jnp.array([[0.1, 0.3]]),
    "C": jnp.array([[0.0, 1.0]])
}
print("\nExp 2 – weak B:")
print(model(inputs_exp2))


Exp 2 – weak B:
[[[0.9 0.9]
  [1.  1. ]]]


Experiment 3: high uncertainty in A

In [13]:
inputs_exp3 = {
    "A": jnp.array([[0.4, 0.9]]),
    "B": jnp.array([[0.8, 0.95]]),
    "C": jnp.array([[0.0, 1.0]])
}
print("\nExp 3 – high uncertainty in A:")
print(model(inputs_exp3))


Exp 3 – high uncertainty in A:
[[[0.6 0.7]
  [1.  1. ]]]


## ***Summary***

- JLNN converts symbolic rules to a differentiable NNX graph
- Inference returns the interval [L, U] – not just one value
- Width of U - L shows the degree of uncertainty / ignorance
- No model parameters were trained – everything is manual